In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from control_factory import get_control_sim
from ch6_scenario import Ch6Scenario

In [ ]:
from ball_control import BallControl
from state_update_model import StatePosition

In [ ]:
async def move_ball(bc: BallControl, src: StatePosition, dest: StatePosition, claw_index: int = 0):
    rel_x = src.x - bc.get_position(claw_index=claw_index).x
    rel_y = src.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index),
        bc.open_claw(claw_index=claw_index))
    await bc.close_claw(claw_index=claw_index)
    
    rel_x = dest.x - bc.get_position(claw_index=claw_index).x
    rel_y = dest.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index))
    await bc.open_claw(claw_index=claw_index)

async def move_ball_by_column(bc: BallControl, src_x: int, dest_x: int, claw_index: int = 0):
    src_column_top_occupied_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == src_x],default=bc.get_state().max_y)
    dest_column_top_vacant_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == dest_x],default=bc.get_state().max_y + 1) - 1
    await move_ball(bc=bc, src=StatePosition(x=src_x, y=src_column_top_occupied_y), dest=StatePosition(x=dest_x, y=dest_column_top_vacant_y), claw_index=claw_index)

In [ ]:
async def challenge6_solution():
    bc = get_control_sim(delay_multiplier=0.3)
    await bc.set_scenario(Ch6Scenario())
    
    await asyncio.gather(
        move_ball_by_column(bc=bc, src_x=0, dest_x=2, claw_index=0),
        move_ball_by_column(bc=bc, src_x=4, dest_x=3, claw_index=1),
    )

    await asyncio.gather(
        bc.move_horizontally(-1, claw_index=0),
        move_ball_by_column(bc=bc, src_x=2, dest_x=4, claw_index=1),
    )
    
    await move_ball_by_column(bc=bc, src_x=3, dest_x=2, claw_index=1)
    
    await asyncio.gather(
        bc.move_horizontally(1, claw_index=1),
        move_ball_by_column(bc=bc, src_x=2, dest_x=0, claw_index=0)
    )    

In [ ]:
await challenge6_solution()